In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,constraints

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from matplotlib import pyplot as plt
import cv2
from libs import archanjo as arch

In [2]:
categorias = [0,1]
categorias_string = ["Hipertensos_Repouso_65_75_Supino","Saudaveis_Repouso_20_30_Supino"]

path_base = "/home/rafael/Documentos/FACOM/Mestrado/Materias/Defesa/Pesquisa/DSP/DataSet_VFC/New_Set/data/"

data_frame_cat = []
data_frame_numpy= []

for x in categorias:
    for class_data_dict in arch.coleta_dados(path_base + categorias_string[x]):
        name_user = class_data_dict[0]
        data = class_data_dict[1]
        sem_outliers = arch.removeoutlier(data.copy(), debug=False)
        estacionario = arch.select_estavel(sem_outliers)

        for data_ in estacionario:
            data_frame_cat.append(x)
            array_normalizado = arch.normalize(data_[1],0,255).astype(np.uint8)
            data_frame_numpy.append(array_normalizado)

data_frame_cat = np.array(data_frame_cat,dtype=np.int)
data_frame_numpy= np.array(data_frame_numpy,dtype=np.int)


In [3]:

rus = RandomUnderSampler(random_state=130,sampling_strategy = "majority"

)
train_images_balance_s,train_labels_balance_s, = rus.fit_resample(data_frame_numpy,data_frame_cat)


In [4]:


train_images_balance, teste_images_predic, train_labels_balance, test_labels_predic = train_test_split(train_images_balance_s, train_labels_balance_s, shuffle=True, random_state=10, train_size=0.8)


print(train_labels_balance)
print(test_labels_predic)

print(train_labels_balance.shape)
print(test_labels_predic.shape)

[0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 1 0 1
 1 1 0 0 1 0 0 1 0]
[1 0 1 0 0 0 1 0 0 0 1 1]
(46,)
(12,)


In [ ]:
data_x=[]
data_y=[]
value_list=64

for index_data in range(0,len(train_images_balance)):
    print(index_data)
    plot_list=arch.sintetiza_RR(train_images_balance[index_data],value_list=value_list,engine_value=100)
    for d_x in plot_list:
        list_ciruclar=arch.get_circular(np.array(d_x))
        for ar_cl in list_ciruclar :
            data_=[]
            for x in range(0,256):
                data_.append(ar_cl)
            data_x.append(np.array(data_))
            data_y.append(train_labels_balance[index_data])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:

train_labels_balance = np.array(data_y,dtype=np.int)
train_images_balance = np.array(data_x,dtype=np.int)

print(train_labels_balance)
print(train_images_balance.shape)
print(train_labels_balance.shape)


In [ ]:
dimensao=(-1,256,256,1)
train_images_balance_2d=train_images_balance.reshape(dimensao)/255

teste_images_predic=teste_images_predic.reshape(dimensao)/255

print(teste_images_predic.shape)
print(test_labels_predic.shape)
print(train_images_balance.shape)
print(train_labels_balance.shape)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(train_images_balance,train_labels_balance , shuffle=True, random_state=120, train_size=0.8)

print(train_labels)
print(test_labels)


print(train_images.shape)

print(test_images.shape)


In [ ]:
model = models.Sequential()


model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(16, 16, 1)))
model.add(layers.Conv2D(32, (5, 5), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(64, (5, 5), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Dropout(0.5))

# model.add(layers.Conv2D(32, (5, 5), activation='relu'))
#
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(32, (5, 5), activation='relu'))
# model.add(layers.Conv2D(32, (5, 5), activation='relu'))

# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (1, 1), activation='relu'))

model.add(layers.Flatten())
# model.add(layers.Dense(128  , activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True),  metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5,
                    validation_data=(test_images, test_labels))

In [ ]:
y_pred=model.predict(teste_images_predic)
y_pred_max = np.argmax(y_pred, axis=1)
print(y_pred)
print(y_pred_max)
print(test_labels_predic)
print(classification_report(test_labels_predic, y_pred_max))


### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


In [ ]:
test_loss, test_acc = model.evaluate(teste_images_predic,  test_labels_predic, verbose=2)
print(test_acc)
